In [1]:
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import os
import cv2
import string
import numpy as np

#Init main values
symbols = string.ascii_lowercase + "0123456789" # All symbols captcha can contain
num_symbols = len(symbols)
img_shape = (50, 200, 1)

Using TensorFlow backend.


In [12]:
# Define a function that creates a net
def create_net():
    img = layers.Input(shape=img_shape) # Get image as an input and process it through some Convs
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 25x7
    
    # Get flattened vector and make 5 branches from it. Each branch will predict one letter
    flat = layers.Flatten()(mp3)
    outs = []
    for _ in range(5):
        dens1 = layers.Dense(1, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='sigmoid')(drop)

        outs.append(res)
    
    # Compile model and return it
    model = Model(img, outs)
    model.compile('rmsprop', loss=['categorical_crossentropy', 'categorical_crossentropy',
                                   'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'])
    return model

In [13]:
# First we need to preprocess the data
def preprocess_data():
    n_samples = len(os.listdir('./images_sample/jpg'))
    X = np.zeros((n_samples, 50, 200, 1))
    y = np.zeros((5, n_samples, num_symbols))

    for i, pic in enumerate(os.listdir('./images_sample/jpg')):
        # Read image as grayscale
        img = cv2.imread(os.path.join('./images_sample/jpg', pic), cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        if len(pic_target) < 6:
            # Scale and reshape image
            img = np.random.randint(0, 256, 10000)
            #img = img / 255
            img = np.reshape(img, (50, 200, 1))
            
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((5, num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
            X[i] = img
            y[:, i] = targs
    
    # Return final data
    return X, y

X, y = preprocess_data()
X_train, y_train = X[:100], y[:, :100]
X_test, y_test = X[100:], y[:, 100:]

In [14]:
# Create net and fit
net = create_net()
history = net.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=32, epochs=15, validation_split=0.2)

Train on 80 samples, validate on 20 samples
Epoch 1/15
80/80 [==============================] - 4s 55ms/step - loss: 0.2118 - dense_32_loss: 0.0747 - dense_34_loss: 0.0747 - dense_36_loss: 0.0747 - dense_38_loss: 0.0747 - dense_40_loss: 0.0544 - val_loss: 0.0000e+00 - val_dense_32_loss: 0.0000e+00 - val_dense_34_loss: 0.0000e+00 - val_dense_36_loss: 0.0000e+00 - val_dense_38_loss: 0.0000e+00 - val_dense_40_loss: 0.0000e+00
Epoch 2/15
80/80 [==============================] - 0s 4ms/step - loss: 0.2238 - dense_32_loss: 0.0373 - dense_34_loss: 0.0373 - dense_36_loss: 0.0373 - dense_38_loss: 0.0373 - dense_40_loss: 0.0373 - val_loss: 0.0000e+00 - val_dense_32_loss: 0.0000e+00 - val_dense_34_loss: 0.0000e+00 - val_dense_36_loss: 0.0000e+00 - val_dense_38_loss: 0.0000e+00 - val_dense_40_loss: 0.0000e+00
Epoch 3/15
80/80 [==============================] - 0s 4ms/step - loss: 0.2097 - dense_32_loss: 0.0373 - dense_34_loss: 0.0373 - dense_36_loss: 0.0373 - dense_38_loss: 0.0373 - dense_40_loss:

In [25]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE) / 255.
    res = np.array(net.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
    return capt, sum(probs) / 5

In [26]:
# Check model on some samples
net.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])

print(predict('./images_sample/jpg/798989_39E93DF4-FC4E-48BB-8D55-4231A3477B47.jpeg'))
print(predict('./images_sample/jpg/796136_181680-AW.jpg'))
print(predict('images_sample/jpg/993039_Mughal Mustard Oil Label 400ml.jpg'))


# print(predict('../input/samples/samples/8n5p3.png'))
# print(predict('../input/samples/samples/f2m8n.png'))
# print(predict('../input/samples/samples/dce8y.png'))
# print(predict('../input/samples/samples/3eny7.png'))
# print(predict('../input/samples/samples/npxb7.png'))

337/337 [==============================] - 0s 1ms/step


ValueError: Error when checking input: expected input_4 to have shape (50, 200, 1) but got array with shape (665, 755, 1)